#  Submit GW for molecules

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA/AiiDAlab imports imports.
%aiida
from aiida.plugins import WorkflowFactory
import aiidalab_widgets_base as awb

# Custom imports.
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer
from apps.surfaces.widgets.import_cdxml import CdxmlUpload2GnrWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
Cp2kMoleculeOptGwWorkChain = WorkflowFactory('nanotech_empa.cp2k.mol_opt_gw')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.StructureUploadWidget(title="Import from computer"),
        awb.SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computer_code_dropdown = awb.CodeDropdown(input_plugin='cp2k', path_to_root="../../")

In [ ]:
#gw_type
gw_type = ipw.Dropdown(
    options=['gpw_std', 'gapw_std', 'gapw_hq'],
    value='gpw_std',
    description='Protocol:',
    disabled=False,
)


#Multiplicity
multiplicity = ipw.IntText(
    value=1,
    description='Multiplicity',
    disabled=False
)
#UKS
uks = ipw.Checkbox(
    value=False,
    description='UKS',
    disabled=False,
    indent=False
)

#spin guess
spins_up = ipw.Text(
    value='',
    placeholder='1 2 10..13',
    description='Spins U:',
    disabled=False
)

spins_down = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Spins D:',
    disabled=False
)

#geometry mode
geo_opt = ipw.Checkbox(
    value=True,
    description='Do geo opt',
    disabled=False,
    indent=False
)

#description
description = ipw.Text(
    value='',
    placeholder='Calculation description',
    description='Description:',
    disabled=False,
    style={"description_width": "initial"}
)
#max nodes
max_nodes = ipw.IntText(
    value=2048,
    description='Max # compute nodes',
    disabled=False,
    style={"description_width": "initial"}
)
#walltime
walltime = ipw.IntText(
    value=3600,
    description='Walltime seconds',
    disabled=False,
    style={"description_width": "initial"}
)


In [ ]:
def get_builder_gw():
    
    builder = Cp2kMoleculeOptGwWorkChain.get_builder()

    builder.metadata.description = description.value
    builder.code = computer_code_dropdown.selected_code
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if uks.value:
        for i in awb.utils.string_range_to_list(spins_up.value)[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(spins_down.value)[0]:
            mag_list[i] = -1 
            
        builder.multiplicity = Int(multiplicity.value)
        
        
    builder.structure = structure_selector.structure_node #StructureData(ase=ase_geom)
    builder.magnetization_per_site = List(list=mag_list)

    builder.protocol = Str(gw_type.value)


    builder.geo_opt = Bool(geo_opt.value)

    debug = 'DEBUG' in description.value
    builder.debug = Bool(debug)
    maxnodes = max_nodes.value
    if debug:
         maxnodes = 1        
    builder.walltime_seconds = Int(walltime.value)
    builder.max_nodes = Int(maxnodes)    
    
    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None



    
btn_submit_gw = awb.SubmitButtonWidget(Cp2kMoleculeOptGwWorkChain, 
                                input_dictionary_function=get_builder_gw,
                                   path_to_root="../../"
                               )
btn_submit_gw.btn_submit.disabled=True

btn_submit_gw.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit_gw.btn_submit.disabled=False
    #check system
    only_one_molecule = empa_viewer.details['system_type'] == 'Molecule'  
    only_one_molecule = only_one_molecule and  len(empa_viewer.details['all_molecules'])==1    
    msg = 'GW for this system not implemented'
    if only_one_molecule:
        btn_submit_gw.btn_submit.disabled=False  
        msg = ''
    spins_up.value = awb.utils.list_to_string_range(empa_viewer.details['spins_up'])
    spins_down.value = awb.utils.list_to_string_range(empa_viewer.details['spins_down'])
    with output:
        clear_output()
        print(msg)
        if uks.value:
            to_display =  [computer_code_dropdown,gw_type,geo_opt,
                           uks,spins_up,spins_down,multiplicity,description,max_nodes,walltime,btn_submit_gw]
        else:
            to_display =  [computer_code_dropdown,gw_type,geo_opt,
                           uks,description,max_nodes,walltime,btn_submit_gw]
        display(ipw.VBox(to_display))


structure_selector.observe(update_all, names='structure')
uks.observe(update_all, names='value')

In [ ]:
display(output)